In [3]:
import numpy as np
import pandas as pd
import random
from bs4 import BeautifulSoup
import urllib2

In [4]:
soup = BeautifulSoup(urllib2.urlopen('http://www.hittrackeronline.com/index.php?perpage=10000', timeout = 60).read(), 'html.parser') # read-in website
print "MLB HR soup created"

MLB HR soup created


In [5]:
my_table = soup('table')[16] # table 16 has 2015 season hrs 
rows = my_table.findChildren(['tr'])
print "The MLB HR data has", len(rows), "rows"

The MLB HR data has 4912 rows


In [6]:
data = {'hitter': [], # hitter name
        'pitcher': [], # pitcher name
        'dist': [], # hr dist
        'speed': [], # speed off bat
        'apex': [], # hit apex
        'elev': [], # elevation angle
        'horiz': [] # horizontal angle
        }

keys = ['hitter', 'pitcher', 'dist', 'speed', 'apex', 'elev', 'horiz'] # need this because keys data.keys() prints in strange order
cols = [3, 5, 10, 11, 14, 12, 13]

size = 50 # set sample size
sample = random.sample(xrange(3, len(rows)-2), size)

#for i in sample: # get a subset of the data
for i in range(3,len(rows)-2): # get the full dataset
    x=rows[i]
    for key, col in zip(keys, cols):
        data[key].append(x.findAll('td')[col])
            #data[key].append(soup('table')[16].findAll('tr')[i].findAll('td')[col])


In [5]:
# cleanup data
floats = ['apex', 'dist', 'elev', 'horiz', 'speed'] # floats
for i in range(0,len(data['hitter'])):
    for key in keys:
        data[key][i] = data[key][i].get_text()
        data[key][i] = data[key][i].strip()
        data[key][i] = data[key][i].encode('ascii','ignore')
    for flo in floats:
        data[flo][i] = float(data[flo][i])

In [6]:
# change names to First Last
data['hitter'] = [h.split(',')[1] + " " + h.split(',')[0] for h in data['hitter']]
data['pitcher'] = [p.split(',')[1] + " " + p.split(',')[0] for p in data['pitcher']]

In [7]:
df = pd.DataFrame(data)
print "The number of missings is", df.count()["hitter"] - df[df.dist > 0].count()["hitter"] 
df = df[df.dist != 0] # remove row if dist == 0

The number of missings is 2


In [10]:
def flip_horiz(row):
    if row['horiz'] > 90:
        return row['horiz'] - 2*(row['horiz'] - 90)
    elif row['horiz'] < 90:
        return row['horiz'] + 2*(90 - row['horiz'])
    else:
        return 90

In [11]:
df.apply (lambda row: flip_horiz(row),axis=1)

0      69.1
1      95.6
2      90.5
3      86.8
4      73.6
5      90.3
6      92.4
7      96.7
8      75.7
9      77.5
10    113.1
11     86.3
12    109.3
13     94.3
14     75.8
...
2904     73.9
2905     96.6
2906     75.1
2907     62.3
2908     73.8
2909     70.7
2910    121.6
2911    100.6
2912    110.9
2913     71.6
2914    118.4
2915    111.1
2916     64.3
2917    125.1
2918     54.3
Length: 2917, dtype: float64

In [12]:
df['horiz_flipped'] = df.apply (lambda row: flip_horiz(row),axis=1)

In [15]:
df.to_csv("hr_data_2015.csv", index=False)

In [16]:
print "hr_data.csv updated"

hr_data.csv updated
